In [1]:
import tensorflow as tf
from tensorflow import estimatormator
from sklearn.datasets import load_wine

In [2]:
vino = load_wine()

In [3]:
print(vino['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [4]:
caracteristicas = vino['data']
objetivo = vino['target']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(caracteristicas, objetivo, test_size=0.3)

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
normalizador = MinMaxScaler()

In [9]:
x_train_normalizado = normalizador.fit_transform(x_train)
x_test_normalizado = normalizador.transform(x_test)

In [10]:
x_train_normalizado

array([[0.68157895, 0.88259958, 0.53225806, ..., 0.3495935 , 0.28571429,
        0.19400856],
       [0.58947368, 0.74213836, 0.48387097, ..., 0.08943089, 0.10622711,
        0.39728959],
       [0.27631579, 0.22851153, 0.51612903, ..., 0.48780488, 0.36630037,
        0.14407989],
       ...,
       [0.64736842, 0.59748428, 0.44623656, ..., 0.26829268, 0.2014652 ,
        0.21540656],
       [0.59736842, 0.20545073, 0.41935484, ..., 0.43902439, 0.54945055,
        0.71825963],
       [0.73947368, 0.70859539, 0.5483871 , ..., 0.07317073, 0.13553114,
        0.14407989]])

In [11]:
x_train_normalizado.shape

(124, 13)

In [12]:
columnas_caracteristicas = [tf.feature_column.numeric_column('x',shape=[13])]

In [14]:
modelo = estimator.DNNClassifier(hidden_units=[20,20,20], feature_columns=columnas_caracteristicas, n_classes=3, optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb447c966d8>, '_task_type': 'worker', '_tf_random_seed': None, '_train_distribute': None, '_num_ps_replicas': 0, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpq2ahosj9', '_device_fn': None, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_is_chief': True, '_log_step_count_steps': 100, '_evaluation_master': '', '_service': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_keep_checkpoint_max': 5}


In [15]:
funcion_entrada = estimator.inputs.numpy_input_fn(x={'x':x_train_normalizado}, y = y_train, shuffle=True, batch_size=10, num_epochs=10)

In [16]:
modelo.train(input_fn=funcion_entrada, steps=600)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpq2ahosj9/model.ckpt.
INFO:tensorflow:loss = 11.575092, step = 1
INFO:tensorflow:global_step/sec: 108.288
INFO:tensorflow:loss = 1.5328377, step = 101 (0.938 sec)
INFO:tensorflow:Saving checkpoints for 124 into /tmp/tmpq2ahosj9/model.ckpt.
INFO:tensorflow:Loss for final step: 0.30901495.


In [17]:
funcion_evaluacion = estimator.inputs.numpy_input_fn(x={'x':x_test_normalizado}, shuffle=False)

In [18]:
predicciones = list(modelo.predict(input_fn=funcion_evaluacion))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpq2ahosj9/model.ckpt-124
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
predicciones_finales = [p['class_ids'][0] for p in predicciones]

In [20]:
predicciones_finales

[1,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0]

In [21]:
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
print(classification_report(y_test, predicciones_finales))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97        19
           1       1.00      0.96      0.98        25
           2       1.00      1.00      1.00        10

   micro avg       0.98      0.98      0.98        54
   macro avg       0.98      0.99      0.98        54
weighted avg       0.98      0.98      0.98        54

